# Tutorial: Working with this stactools subpackage

Stactools ([docs](https://stactools.readthedocs.io/en/latest/), [source](https://github.com/stac-utils/stactools)) is a command line tool and library for working with [STAC](https://stacspec.org/), based on [PySTAC](https://github.com/stac-utils/pystac). [Stactools packages](https://github.com/stactools-packages) are add-ons for stactools that provide STAC functionality for specific datasets, such as [Sentinel 2](https://github.com/stactools-packages/sentinel2) and [Landsat](https://github.com/stactools-packages/landsat). 

Stactools and its packages can be accessed from the CLI or from within normal Python code. This notebook provides examples of both.

## 1. Installing stactools

To use a package, first install `stactools` then the package. `stactools` can be installed with `pip`.

In [ ]:
%pip install stactools

Check that the `stac` CLI tool is installed

In [ ]:
!stac

Notice the Commands available. Depending on your current environment, you may already have one or more dataset commands available (e.g. `lilahkhglacier`). In the next step we will demonstrate how to add a dataset command.

## 2. Installing a stactools package

Here we'll use the subpackage that this notebook resides within, but feel free to change the `PACKAGE` variable to any one of the subpackage repo names in the [stactools-subpackages](https://github.com/stactools-packages). Other examples include `sentinel2`, `planet`, and `landsat`.

In [ ]:
PACKAGE = "lila-hkh-glacier"

In [ ]:
%pip install stactools-{PACKAGE}

Notice the addition of the subpackage command in stactools now.

In [ ]:
!stac

## 3. Using the stactools subpackage

You can now explore the STAC package commands to ingest and describe the data

In [ ]:
!stac lilahkhglacier --help

The `lilahkhglacier` command contains:
- `create-cog` command which will convert a GeoTiff to Cloud-optimized GeoTiff format
- `lilahkhglacier-fused` subcommand
- `lilahkhglacier-slice` subcommand

The LILA HKH Glacier dataset contains two main types of raster data:
- `fused`: images we refer to as fused are large SRTM and Landsat 7 images, each with 15 bands in .tif format
- `slice`: images we refer to as slices are 512 x 512 pixel tiles derived from fused images, each with 15 bands in .npy format, and an associated 2-band mask, also in .npy format.

In [ ]:
!stac lilahkhglacier lilahkhglacier-fused --help
!echo "========"
!stac lilahkhglacier lilahkhglacier-slice --help

And more specific help with an individual command.

In [ ]:
!stac lilahkhglacier lilahkhglacier-slice create-items --help

So far we've used Jupyter Notebooks' IPython [line magic](https://ipython.readthedocs.io/en/stable/interactive/magics.html) to work with stactools packages on the command line, but it's also possible to use them within Python scripts.

In [ ]:
from stactools.lila_hkh_glacier import stac, cog, utils

tif_path = "../tests/data-files/raster_data/LE07_134040_20070922_clip.tif"
output_path = "LE07_134040_20070922_cog.tif"

# convert a geotiff to cloud-optimized geotiff
cog.create_cog(tif_path, output_path)

In [ ]:
output_dir = "."
fuseddir = "../tests/data-files/raster_data/"

# create a STAC collection that reflects the contents of a directory of fused images
stac.create_fused_collection(output_dir, fuseddir)

In [ ]:
# create a STAC item for the COG we created above
stac.create_fused_item(output_path, output_dir)

In [ ]:
import json

metadata_path = "../tests/data-files/slices.geojson"
slicedir = "../tests/data-files/slices"

with open(metadata_path, "r") as f:
    metadata_dict = json.load(f)
    # update paths to reflect the current location of the files
    metadata_dict = utils.update_metadata_paths(metadata_dict, slicedir)

# create a STAC collection that reflects the features within a metadata file
stac.create_slice_collection(metadata_dict, output_dir)

In [ ]:
feature = metadata_dict["features"][0]
epsg_code = utils.get_epsg(metadata_dict)

# create a STAC item for one of the features in the metadata file
slice_item = stac.create_slice_item(feature, output_dir, epsg_code)
slice_item

In [ ]:
import numpy as np

# load the contents of the assets in the item
raster_content = np.load(slice_item.assets["raster"].href)
raster_labels_content = np.load(slice_item.assets["raster_labels"].href)

raster_content.shape, raster_labels_content.shape

At this point, `raster_content` and `raster_labels_content` are image arrays, which you could view through a plotting dependency like matplotlib.